<a href="https://colab.research.google.com/github/callmesukhi/BFVM19PROG1/blob/main/assessments/Assignment_week_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Week 03 Assignment Covid

Welcome to **week three** of this course programming 1. You will learn about combining data with pandas and numpy and you will learn to visualize with bokeh. Concretely, you will preprocess the partly Synthetic Covid data in an appropiate format in order to conduct statistical and visual analysis. Learning outcomes:

- Load a tabular dataset
- Inspect the dataset for quality and metadata information
- Combine data from several tables into one dataframe
- Subselect specific data from dataframes
- Reshape the dataset into a format suitable for visual and statistical analysis
- Visualize data using bokeh 
- Use widgets to make the plot interactive (optional)
- Use geomap to plot locations (optional)


Your job is to **visualize the lab values taken for COVID-19 patients of survived versus not survived patients**. 

The assignment consists of 6 parts:

- [part 1: load the data](#0)
     - [Exercise 1.1](#ex-11)
- [part 2: data wrangling](#1)
     - [Exercise 2.1](#ex-21)
- [part 3: more wrangling](#2)
     - [Exercise 3.1](#ex-31)
- [part 4: plot the data](#3)
     - [Exercise 4.1](#ex-41)
- [part 5: plot patient location](#5)
     - [Exercise 5.1](#ex-51)


Part 1 and 4 are mandatory, part 5 is optional (bonus)
To pass the assingnment you need to a score of 60%. 


## About the data

The data is generated by Synthea's COVID-19 module. The data was constructed using three peer-reviewed publications published in the early stages of the global pandemic, when less was known, along with emerging resources, data, publications, and clinical knowledge. The simulation outputs synthetic Electronic Health Records (EHR), including the daily consumption of Personal Protective Equipment (PPE) and other medical devices and supplies. For this assignment the `conditions`, `patients`, `observations`, `careplans` and `encounters` table will be used. The Data is stored in separate tables to avoid redundancy, with as a concequence that tables need to be combined and reorganized in dataframes for analysing purpose.

Source: Walonoski J, Klaus S, Granger E, Hall D, Gregorowicz A, Neyarapally G, Watson A, Eastman J. Synthea™ Novel coronavirus (COVID-19) model and synthetic data set. Intelligence-Based Medicine. 2020 Nov;1:100007. https://doi.org/10.1016/j.ibmed.2020.100007

Please <a href = "https://storage.googleapis.com/synthea-public/10k_synthea_covid19_csv.zip">download</a> the data

#### Covid Patients
Patients are considered Covid patients if they are identified with `CODE` `840539006`


#### Survivors
Patients that had covid and where tested negative after isolation have tested code `94531-1`,  SARS-CoV-2 RNA Pnl Resp NAA+probe (covid-sars test) + a value of `Not detected (qualifier value)`. These patients are considered to be survived covid patients. 

#### Non-Survivors
Patients that did not survived Covid have a `DEATHDATE` which is not null. 


#### Lab values  COVID-19 patients

Patients are monitored for blood and heart conditions once they are admitted in Hospital or under treatment. The lab values of interest are as follow: 

- `48065-7`  Fibrin D-dimer FEU [Mass/volume] in Platelet poor plasma
- `26881-3`   Interleukin 6 [Mass/volume] in Serum or Plasma
- `2276-4` Ferritin [Mass/volume] in Serum or Plasma
- `89579-7` Troponin I.cardiac [Mass/volume] in Serum or Plasma by High sensitivity method
- `731-0` Lymphocytes [#/volume] in Blood by Automated count
- `14804-9` Lactate dehydrogenase [Enzymatic activity/volume] in Serum or Plasma by Lactate to pyruvate reaction


<a name='0'></a>
## Part 1: Load the data (20 pt)

Instructions: Load the data of the following files

- conditions.csv
- patients.csv
- observations.csv
- careplans.csv
- encounters.csv

Get yourself familiar with the data. Create some meaningful overviews. Answer the following questions

1. How many patients are there
2. How many covid-patients are there
3. How many patients do have a 'Hospital admission for isolation' encounter
    
<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
    <ul><li>use a unique dataframe for each file, use a meaningful name</li>
    <li>pandas.read_csv() method can be used to read a csv file</li>
    <li>pandas.DataFrame.head() method is often used to inspect the dataframe</li>
    <li>.unique() returns a list of unique values of a column</li>
</ul>
</details>

<a name='ex-11'></a>
### 1.1 Code your solution

In [49]:
import pandas as pd
import numpy as np

In [50]:
# loading the data 
conditions = pd.read_csv('/content/conditions.csv')
patients = pd.read_csv('/content/patients.csv')
observations = pd.read_csv('/content/observations.csv')
careplans = pd.read_csv('/content/careplans.csv')
encounters = pd.read_csv('/content/encounters.csv')

In [51]:
num_pat = len(patients) # finding the total number of patients 
num_cov = len(conditions.query('CODE == "840539006"')) # finding the total covid patients 
num_admitted = len(encounters[encounters.CODE == 1505002]) # finding the admitted patients 
num_died = len(patients.DEATHDATE) - patients.DEATHDATE.isnull().sum() # finding the dead patients 

### 1.2 Test your solution
The following function needs to be called. You can use this as a test. There are however more meaningful overviews 
you can create. 

In [52]:
def part1(num_pat, num_cov, num_admitted, num_died):
    print(f'There are {num_pat} patients in total')
    print(f'There are {num_cov} covid patients')
    print(f'There are {num_admitted} admitted patients')
    print(f'{num_died} patients died')

part1(num_pat, num_cov, num_admitted, num_died)

There are 12352 patients in total
There are 8820 covid patients
There are 1867 admitted patients
2352 patients died


### Expected outcome

There are 12352 patients in total
There are 8820 covid patients
There are 1867 admitted patients
2352 patients died

---

<a name='1'></a>
## Part 2: Data Wrangling: set up the dataframe (30 pt)

In this part we are going to combine data to create a dataframe with values of interest for the lab values analysis. 

We would like a dataframe containing the following information per record (only Covid patients!!!)

- `PATIENT` - the ID of the covid patient
- `days` - the number of days the patient is under observation
- `CODE-Y` - the code of the observation  
- `VALUE` - the lab value of the observation

where only the following observation codes needs to be selected:

- `48065-7`  Fibrin D-dimer FEU [Mass/volume] in Platelet poor plasma
- `26881-3`   Interleukin 6 [Mass/volume] in Serum or Plasma
- `2276-4` Ferritin [Mass/volume] in Serum or Plasma
- `89579-7` Troponin I.cardiac [Mass/volume] in Serum or Plasma by High sensitivity method
- `731-0` Lymphocytes [#/volume] in Blood by Automated count
- `14804-9` Lactate dehydrogenase [Enzymatic activity/volume] in Serum or Plasma by Lactate to pyruvate reaction

The days information is not primarely available and needs to be calculated by substracting observation DATE - START. 

An example of such a dataframe is given below:

                                      days   CODE-Y  VALUE    UNITS
PATIENT                                                            
00079a57-24a8-430f-b4f8-a1cf34f90060   0.0  14804-9  241.8      U/L
00079a57-24a8-430f-b4f8-a1cf34f90060   0.0   2276-4  489.5     ug/L
00079a57-24a8-430f-b4f8-a1cf34f90060   0.0  48065-7    0.4    ug/mL
00079a57-24a8-430f-b4f8-a1cf34f90060   0.0    731-0    1.1  10*3/uL
00079a57-24a8-430f-b4f8-a1cf34f90060   0.0  89579-7    3.4    pg/mL
00079a57-24a8-430f-b4f8-a1cf34f90060   1.0    731-0    1.1  10*3/uL
00079a57-24a8-430f-b4f8-a1cf34f90060   2.0  14804-9  233.7      U/L
00079a57-24a8-430f-b4f8-a1cf34f90060   2.0   2276-4  497.3     ug/L
00079a57-24a8-430f-b4f8-a1cf34f90060   2.0  48065-7    0.3    ug/mL
00079a57-24a8-430f-b4f8-a1cf34f90060   2.0    731-0    1.0  10*3/uL

In [ ]:
#Possible approach:

#Select all the patients with covid from the conditions table
#Combine conditions table (only covid patients) with the patient table into a covid_patient table
#select the only the relevant lab observations from the observations table into a lab_obs table
#merge the covid_patient table with the lab_obs table into a covid_patients_obs table
#clean the covid_patients_obs table (rename columns, select only relevant columns, sort, typecast, add days column)

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
    <ul><li>you can use pandas.DataFrame.merge() to merge dataframes</li>
    <li>df = df[(df.CODE == condition1 | df.CODE == condition1 )] selects rows with CODE of 2 conditional values</li>
    <li>df.DATE - df.START return days if DATE and START are datetime format</li>
    <li>pd.to_datetime() can be used to typecast to datetime</li>
</ul>
</details>

<a name='ex-21'></a>
### 2.1 Code your solution

In [53]:
# extracting all the covid patients from conditions df 
covid_pat = conditions.query('CODE == "840539006"')
# merging the covids patients df with patients df into covid_patient df
covid_patient = covid_pat.merge(patients, how='left', left_on='PATIENT', right_on='Id')

In [54]:
# selecting only relevent lab observations from obversations table into lab_obs table 
lab_obs = observations[observations.CODE.isin(['48065-7','26881-3','2276-4','89579-7','731-0','14804-9'])]
# keeping only relevent columns
covid_patient = covid_patient[['START', 'STOP', 'PATIENT', 'CODE', 'Id']]
# merging covid_patient table with lab_obs table 
covid_patients_obs = covid_patient.merge(lab_obs, on='PATIENT')
# Cleaning the covid_patients_obs table
covid_patients_obs['DATE'] = pd.to_datetime(covid_patients_obs['DATE'])
covid_patients_obs['START'] = pd.to_datetime(covid_patients_obs['START'])
covid_patients_obs['DAYS'] = (covid_patients_obs.DATE - covid_patients_obs.START) / np.timedelta64(1, 'D')
covid_patients_obs['VALUE'] = pd.to_numeric(covid_patients_obs['VALUE'], errors='coerce')

---

<a name='2'></a>
## Part 3: Data Wrangling, split into survived and not survived (10 pt)

Now we have the required data we would like to split the data into survived and not survived. First we fetch all the ids of the survived and deceased patients. We can use these ids to select the records of the survived patients and the patients that did not survived

Your job is to split the data into survived and not survived records. There are multiple ways to do this. One way is the  `.isin()` method

In [55]:
#the following code is given, RUN THIS CELL
#get survived and deceased ids
completed_isolation_patients = careplans[(careplans.CODE == 736376001) & (careplans.STOP.notna()) \
                                          & (careplans.REASONCODE == 840539006)].PATIENT
negative_covid_patient_ids = observations[(observations.CODE == '94531-1') \
                                          & (observations.VALUE == 'Not detected (qualifier value)')].PATIENT.unique()
survivor_ids = np.union1d(completed_isolation_patients, negative_covid_patient_ids)
deceased_ids = patients[patients.DEATHDATE.notna()].Id

<a name='ex-31'></a>
### 3.1 Code your solution

In [56]:
survived = len(covid_patients_obs[covid_patients_obs.PATIENT.isin(survivor_ids)])
died = len(covid_patients_obs[covid_patients_obs.PATIENT.isin(deceased_ids)])

### 3.2 Test your solution

In [57]:
def test3(survived, died):
    print(f'patients records survived: {survived}, patients records deceased {died}')

test3(survived, died)

patients records survived: 57303, patients records deceased 16793


#### Expected outcome

patients records survived: 57303, patients records deceased 16793

---

<a name='3'></a>
## Part 4: Plot the data (20 pt)

Create plots with the lab data, for each code one plot. Separate the survivors and the deceased by color. An example of such a plot is given below. You can create 6 plots in one grid (for each code one plot) or use a widget (for instance a drop down menu widget) to select a lab CODE. Plot on the x-axis the days, on the y-axis the VALUE. Use proper labels, titles and legends.

<img src="https://github.com/callmesukhi/BFVM19PROG1/blob/main/images/week3_plot.png?raw=1" width="500" height="500"/>

<a name='ex-41'></a>
### 4.1 Code your solution

In [64]:
from bokeh.plotting import figure, show
from bokeh.models.widgets import Panel,Tabs
from bokeh.io import output_notebook
from bokeh.layouts import gridplot
from bokeh.palettes import magma 
output_notebook()

In [60]:
d_dimer = covid_patients_obs.query('CODE_y == "48065-7"')
survived_pat_d_dimer = d_dimer[d_dimer.PATIENT.isin(survivor_ids)]
died_pat_d_dimer = d_dimer[d_dimer.PATIENT.isin(deceased_ids)]

interleukin_6 = covid_patients_obs.query('CODE_y == "26881-3"')
survived_pat_interleukin_6 = interleukin_6[interleukin_6.PATIENT.isin(survivor_ids)]
died_pat_interleukin_6 = interleukin_6[interleukin_6.PATIENT.isin(deceased_ids)]

ferritin = covid_patients_obs.query('CODE_y == "2276-4"')
survived_pat_ferritin = ferritin[ferritin.PATIENT.isin(survivor_ids)]
died_pat_ferritin = ferritin[ferritin.PATIENT.isin(deceased_ids)]

troponin_i_cardiac = covid_patients_obs.query('CODE_y == "89579-7"')
survived_pat_troponin_i_cardiac = troponin_i_cardiac[troponin_i_cardiac.PATIENT.isin(survivor_ids)]
died_pat_troponin_i_cardiac = troponin_i_cardiac[troponin_i_cardiac.PATIENT.isin(deceased_ids)]

lymphocytes = covid_patients_obs.query('CODE_y == "731-0"')
survived_pat_lymphocytes = lymphocytes[lymphocytes.PATIENT.isin(survivor_ids)]
died_pat_lymphocytes = lymphocytes[lymphocytes.PATIENT.isin(deceased_ids)]

lactate_dehydrogenase = covid_patients_obs.query('CODE_y == "14804-9"') 
survived_pat_lactate_dehydrogenase = lactate_dehydrogenase[lactate_dehydrogenase.PATIENT.isin(survivor_ids)]
died_pat_lactate_dehydrogenase = lactate_dehydrogenase[lactate_dehydrogenase.PATIENT.isin(deceased_ids)]


In [61]:

p1 = figure(plot_width=500, plot_height=500, title="Fibrin D-dimer FEU [Mass/volume] in Platelet poor plasma")
p1.scatter(survived_pat_d_dimer.DAYS, survived_pat_d_dimer.VALUE, color = 'green', legend_label="survived")
p1.scatter(died_pat_d_dimer.DAYS, died_pat_d_dimer.VALUE, color="red", legend_label = 'deceased')
p1.xaxis.axis_label = 'Days'
p1.yaxis.axis_label = 'pg/mL'

p2 = figure(plot_width=500, plot_height=500, title="Interleukin 6 [Mass/volume] in Serum or Plasma")
p2.scatter(survived_pat_interleukin_6.DAYS, survived_pat_interleukin_6.VALUE, color = 'green', legend_label="survived")
p2.scatter(died_pat_interleukin_6.DAYS, died_pat_interleukin_6.VALUE, color="red", legend_label = 'deceased')
p2.xaxis.axis_label = 'Days'
p2.yaxis.axis_label = 'pg/mL'

p3 = figure(plot_width=500, plot_height=500, title="Ferritin [Mass/volume] in Serum or Plasma")
p3.scatter(survived_pat_ferritin.DAYS, survived_pat_ferritin.VALUE, color = 'green', legend_label="survived")
p3.scatter(died_pat_ferritin.DAYS, died_pat_ferritin.VALUE, color="red", legend_label = 'deceased')
p3.xaxis.axis_label = 'Days'
p3.yaxis.axis_label = 'pg/mL'

p4 = figure(plot_width=500, plot_height=500, title="Troponin I.cardiac [Mass/volume] in Serum or Plasma by High sensitivity method")
p4.scatter(survived_pat_troponin_i_cardiac.DAYS, survived_pat_troponin_i_cardiac.VALUE, color = 'green', legend_label="survived")
p4.scatter(died_pat_troponin_i_cardiac.DAYS, died_pat_troponin_i_cardiac.VALUE, color="red", legend_label = 'deceased')
p4.xaxis.axis_label = 'Days'
p4.yaxis.axis_label = 'pg/mL'

p5 = figure(plot_width=500, plot_height=500, title="Lymphocytes [#/volume] in Blood by Automated count")
p5.scatter(survived_pat_lymphocytes.DAYS, survived_pat_lymphocytes.VALUE, color = 'green', legend_label="survived")
p5.scatter(died_pat_lymphocytes.DAYS, died_pat_lymphocytes.VALUE, color="red", legend_label = 'deceased')
p5.xaxis.axis_label = 'Days'
p5.yaxis.axis_label = 'pg/mL'

p6 = figure(plot_width=500, plot_height=500, title="Lactate dehydrogenase [Enzymatic activity/volume] in Serum or Plasma by Lactate to pyruvate reaction")
p6.scatter(survived_pat_lactate_dehydrogenase.DAYS, survived_pat_lactate_dehydrogenase.VALUE, color = 'green', legend_label="survived")
p6.scatter(died_pat_lactate_dehydrogenase.DAYS, died_pat_lactate_dehydrogenase.VALUE, color="red", legend_label = 'deceased')
p6.xaxis.axis_label = 'Days'
p6.yaxis.axis_label = 'pg/mL'


show(gridplot([[p1, p2], [p3, p4], [p5, p6]], plot_width=500, plot_height=500))

<a name='4'></a>
## Part 5: Plot the location of the patients (10 pt)

This is a bonus part. Can you plot the patients location on a map? See also 
https://docs.bokeh.org/en/latest/docs/user_guide/geo.html


<a name='ex-51'></a>
### 5.1 Code your solution